# Task E

## 1. Clear and replace data

In [1]:
import json
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from functools import reduce
from collections import Counter

In [2]:
def find_key_in_dict(keys_i_want, dictionary):
    l = []
    def iterate_through_dict(keys_i_want, dictionary):
        for i in dictionary:
            if i in keys_i_want:
                l.append({i: dictionary[i]})
                keys_i_want.remove(i)
            if type(dictionary[i]) is dict:
                iterate_through_dict(keys_i_want, dictionary[i])
    iterate_through_dict(keys_i_want,dictionary)            
    return l

In [4]:
with open('Oct19thTweetMine.json','r') as f:
#     print(sum([1 for i in f]))
    comments_list =[]
    #big_city = ['Dallas', 'Houston', 'San Antonio', 'Austin', 'El Paso']
    count = 1
    for line in f:
        tweet = json.loads(line)
        if count <= 4000:
            l = find_key_in_dict(['full_text', 'location', 'text'], tweet)
            d = {}
            for i in range(len(l)):
                for j in l[i]:
                    if j == 'location':
                        d['location'] = l[i][j]
                        break
                for j in l[i]:
                    if j == 'full_text':
                        d['text'] = l[i][j]
                        break
                if 'text' not in d.keys():
                    for j in l[i]:
                        if j == 'text':
                            d['text'] = l[i][j]
                            break
            comments_list.append(d)
#             print(count,d)
#             print()
        count += 1
    
comments_list

[{'text': "BREAKING: The Houston Chronicle, the Texas newspaper that endorsed Romney over Obama and both George Bush Sr and Jr when they ran for President, has endorsed Beto O'Rourke over Ted Cruz for the U.S. Senate.\n\nVote @BetoORourke https://t.co/BbzImtIl8Z",
  'location': None},
 {'text': 'Beto O’Rourke is a total lightweight compared to Ted Cruz, and he comes nowhere near representing the values and desires of the people of the Great State of Texas. He will never be allowed to turn Texas into Venezuela!',
  'location': None},
 {'text': "RT @thehill: JUST IN: Houston Chronicle endorses Beto O'Rourke in Texas Senate race https://t.co/2YX0UgCqEc https://t.co/KFqHhmWwRO",
  'location': 'ATL/Houston'},
 {'text': "I'm getting ready to go to a phone bank for @BetoORourke  who is running for the senate in my home state of Texas! If you want to do something for his campaign, it's easy to do even from home (or donate!). Just go to https://t.co/z80ZpBX4Q9",
  'location': 'Brooklyn, NY'},
 

__Replace 10 biggest cities by population (e.g., Dallas, Houston, San Antonio, Austin, El Paso, etc.) with big_city and the rest with small_town. Then merge the location and the text columns. __

In [5]:
# comments=list(map(lambda x: x.lower(), comments))
big_city = ['Dallas', 'Houston', 'San Antonio', 'Austin', 'El Paso','Fort Worth','Arlington','Corpus Christi','Plano','Laredo']
comments = []
for i in comments_list:
    m =str(i['text'])
    l = str(i['location']).split(",")[0]
    if l in big_city:
        l = "big_city"
    else:
         l = "small_town"
    comments.append(l+' '+m)
          

len(comments )



4000

In [6]:
comments=list(map(lambda x: x.lower(), comments))
comments

["small_town breaking: the houston chronicle, the texas newspaper that endorsed romney over obama and both george bush sr and jr when they ran for president, has endorsed beto o'rourke over ted cruz for the u.s. senate.\n\nvote @betoorourke https://t.co/bbzimtil8z",
 'small_town beto o’rourke is a total lightweight compared to ted cruz, and he comes nowhere near representing the values and desires of the people of the great state of texas. he will never be allowed to turn texas into venezuela!',
 "small_town rt @thehill: just in: houston chronicle endorses beto o'rourke in texas senate race https://t.co/2yx0ugcqec https://t.co/kfqhhmwwro",
 "small_town i'm getting ready to go to a phone bank for @betoorourke  who is running for the senate in my home state of texas! if you want to do something for his campaign, it's easy to do even from home (or donate!). just go to https://t.co/z80zpbx4q9",
 "small_town rt @thehill: houston chronicle endorses beto o'rourke in texas senate race https://

## 2. Calculate the lift value

In [7]:
# Number of each type of city mentioned in the tweets
city = {'big_city', 'small_town'}
city_mention = dict()

#initialize counter
for i in city:
    city_mention[i] = []
    
for i in city:
    for j in comments:
        for k in word_tokenize(j):
            if k == i:
                city_mention[i].append(k)
            break
    
    print(i + " was totally mentioned "+str(len(city_mention[i]))+" times.")

#print(city_mention)

big_city was totally mentioned 186 times.
small_town was totally mentioned 3814 times.


In [8]:
#Number of each name mentioned in the tweets
name = {'beto':['beto', 'betoorourke','@betoorourke'], 'cruz':['cruz','tedcruz','@tedcruz' ]}
name_mention = dict()


#initialize counter
for i in name:
    name_mention[i] = []
    
    
for i in name:
    for m in name[i]:
        for j in comments:
             for k in word_tokenize(j):
                     if k == m:
                            name_mention[i].append(j)
                            break                    
    print(i + " was totally mentioned "+str(len(name_mention[i]))+" times.")

beto was totally mentioned 5517 times.
cruz was totally mentioned 2210 times.


In [9]:
# Number of tweets containing each pair of city type and name
#(big_city, beto),(big_city, cruz),(small_town, beto),(small_town,, cruz)

# for each city type mentioned, find number of times each of name was also mentioned
cross_mention = dict()

# init dictionary
for i in city_mention:
    cross_mention[i] = dict()
    for j in name_mention:
        cross_mention[i][j] = [ ]
#{'big_city': {'beto': [], 'cruz': []}, 'small_town': {'beto': [], 'cruz': []}}

for i in city:
    for j in name_mention:
        for k in range(len(name_mention[j])):
            for l in word_tokenize(name_mention[j][k]):
                if l == i:
                    cross_mention[i][j].append( name_mention[j][k])
                    break
        print("Pair "+"("+i+","+j+")" + " was totally mentioned "+str(len(cross_mention[i][j]))+" times.")




Pair (big_city,beto) was totally mentioned 266 times.
Pair (big_city,cruz) was totally mentioned 113 times.
Pair (small_town,beto) was totally mentioned 5251 times.
Pair (small_town,cruz) was totally mentioned 2097 times.


In [10]:
N = len(comments)
N

4000

In [11]:
# now we can calculate the lift value
lift = [ ]

for i in range(len(city_mention.keys())):
    row = []
    for j in range(len(name_mention.keys())):
        row.append( round(N*len(cross_mention[list(city_mention.keys())[i]][list(name_mention.keys())[j]])/((len(city_mention[list(city_mention.keys())[i]]))*(len(name_mention[list(name_mention.keys())[j]]))), 4))

    lift.append(row)
lift

[[1.0369, 1.0996], [0.9982, 0.9951]]

In [12]:
lift_chart = pd.DataFrame(lift, columns=name_mention.keys(), index=city_mention.keys())
lift_chart

,beto,cruz
big_city,1.0369,1.0996
small_town,0.9982,0.9951


## 3. Calculate sentiment score

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [14]:
def cal_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    return snt['compound']

In [15]:
#calculate sentiment score
score = []

#For each combination of each city type and each name(beto or cruz), 
#firstly calculate the sentiment score of each sentence of the combination, then calculate the average of the scores
#as the final sentiment score ot this combination

for i in list(city_mention.keys()):
    row=[]
    for j in list(name_mention.keys()):
        a=0
        for k in cross_mention[i][j]:
             a+=cal_sentiment_scores(k)
        row.append(round(a/len(cross_mention[i][j]),4))
    score.append(row)
            
score
        

[[0.3949, 0.3476], [0.3839, 0.362]]

In [16]:
sentiment_score_chart = pd.DataFrame(score, columns=name_mention.keys(), index=city_mention.keys())
sentiment_score_chart

,beto,cruz
big_city,0.3949,0.3476
small_town,0.3839,0.3620
